Deployed use of SVGnest of flat shapes in a stock area

There is no chance of SVGnest being linked into FC, because it's implemented in Javascript.  However, it can be called externally using an interface defined by the SVG notation, and the original shapes with their new orientations found in the output.  

This is a good thing, because heavy computations should not be done in the same process as FC, because you don't want them bringing the whole thing down when they fail, and they can run asynchronously.  We should take advantage of this indirection.

#### Deployed use of SVGnest of flat shapes in a stock area

Some small annoyances:

Issue 5) Cannot select other solid as stock (it deselects it for me).

Issue 6) If I load a FC file in a script and save it, the Job gets ruined.

Where is the best place to file these sorts of little issues?  The FC bug list discourages putting things in there, and they seem too small. 

------

It would be transformative to have a good nesting feature for use in water jet or laser cutting.  

The natural integration would be make it a feature of the Path workbench and apply it to a Job, because it has both a stock body and a list of bodies to cut.  

I've tried to make a that did this using the nester in arch (announced at ...), but it's too underdeveloped, and probably can't ever be good because it's a minor feature among thousands in FC.  

The only open source nester around is SVGnest, which is written in Javascript, so is never going to be linked in as part of the FC executable.  Therefore it can only be run in another process by passing files to it on the disk.  

This is actually a good thing, because self-contained computationally intensive processes should always be done their own process (or preferably on a completely different machine) so they don't bring down the whole UI when they overflow and crash, and can operate asyncronously.  

So I have written macros that can both (A) export the geometry to an SVG file, and (B) read the output from SVGnest (after you have run it in a browser somewhere else) and copies the positions of each piece into the Placement of the corresponding Body.  

This is barely useable, but a lot better than nothing.  The SVGnest can take any shape as input, (not only rectangles) so you can route your parts around the clamps or holes in the stock.  Perhaps the shape of the stock can be acquired using an over-head camera.  




In [2]:
# Import the modules
import sys
freecadpath = "/home/julian/extrepositories/FreeCAD/freecad-build/lib"
sys.path.append(freecadpath)
import FreeCAD
doc = FreeCAD.open("/home/julian/data/freecad/nestdemo.fcstd")

In [3]:
# Function to extract the flat bottom face of each body
from FreeCAD import Vector
def GetBaseFace(shape):
    basefaces = [ ]
    for face in shape.Faces:
        #print(face.Surface.isPlanar(), face.Surface.Axis, face.Surface.Position)
        if face.Surface.isPlanar():
            if abs(face.Surface.Axis.z) == 1 and face.Surface.Position.z == 0:
                basefaces.append(face)
    assert len(basefaces) == 1, len(basefaces)
    return basefaces[0]


In [4]:
job = doc.Job

stockface = GetBaseFace(job.Stock.Shape)
partfaces = [ ]
for part in job.Model.Group:
    partfaces.append(GetBaseFace(part.Shape))

print(stockface, partfaces)

(<Face object at 0x55d82b5eeb70>, [<Face object at 0x55d82b5e8bf0>, <Face object at 0x55d82ae1d560>, <Face object at 0x55d82b5c8f40>])


## Skip to Option 3 below for the real solution   
# VVV

In [7]:
#
# Option 1, use the Nester in FC (though not as complete as SVGnest)
#
import Arch
nest = Arch.ArchNesting.Nester(stockface, partfaces)
nest

<ArchNesting.Nester instance at 0x7f880bdd3290>

In [35]:
#
# Option 2, use the projection function converted to SVG face 
#
# It uses ProjectionAlgos which calls into HLRBRep_Algo of OCC 
# to do the projection of the shapes (even though they are flat and 
# We don't want to project them).  
# The result of the projection goes to SVGOutput::exportEdges()
# which applies BRepAdaptor_Curve (from OCC) to each edge 
# of the now projected TopoDS_Shape to generates the actual 
# SVG code (printCircle, etc), including styles (which we also don't want).  

import Drawing
x = Drawing.projectToSVG(stockface, Vector(0,0,1))
print(x)

<g   fill="none"
   stroke="rgb(0, 0, 0)"
   stroke-linecap="butt"
   stroke-linejoin="miter"
   stroke-width="1.0"
   transform="scale(1,-1)"
  >
<path id= "1" d=" M 0 0 L 0 30 " />
<path id= "2" d=" M 0 30 L 50 30 " />
<path id= "3" d=" M 50 0 L 50 30 " />
<path id= "4" d=" M 0 0 L 50 0 " />
</g>



In [5]:
#
# Extract the geometry from within the cruft that has been 
# included in this SVG text
# 

import re
def concatsvgpaths(svgtext):
    dlist = [ ]
    lastpt = None
    for ctype, ctext in re.findall('<(path|circle)(.*?)/>', svgtext):
        if ctype == 'path':
            dtext = re.search('d="(.*?)"', ctext).group(1)
            dvals = dtext.split()
            if lastpt is not None and len(dvals) and \
                    dvals[0] == 'M' and dvals[1:3] == lastpt:
                dlist.append(" ".join(dvals[3:]))
            else:
                dlist.append(dtext.strip())
            lastpt = dvals[-2:]
        else:
            assert False, (ctype, "Not implemented")
    return " ".join(dlist)
        
                    

In [34]:
svgstock = concatsvgpaths(Drawing.projectToSVG(stockface, Vector(0,0,1)))
svgparts = [ concatsvgpaths(Drawing.projectToSVG(partface, Vector(0,0,1)))  for partface in partfaces ]

# This almost works
svgstock = 'M 0 0 L 0 30 L 50 30 L 50 0 L 0 0'  # redo as not sequential
fout = open("svgnestinput.svg", "w")
fout.write('<svg xmlns="http://www.w3.org/2000/svg" viewBox="-20 -20 40 40">\n')
fout.write('<path d="%s"/>\n' % svgstock)
for svgpart in svgparts:
    fout.write('<path d="%s"/>\n' % svgpart)
fout.write("</svg>\n")
fout.close()

# now upload to https://svgnest.com/ (which runs in *your* browser, not on their server)

## Option 3

In [6]:
#
# Option 3, use the  
# (even though it's already flat, so we don't want anything projecting)
#
# Path.fromShapes is C-code that ultimately depends on BRepAdaptor_Curve
# But internally orders the Wires so that the paths come out in sequence.
# However, I wish I could get to the raw shapes themselves, before they 
# are unnecessarily converted into this Path Command format.
#
import Path
x = Path.fromShapes(stockface) # which is C-code
(x.Commands)

[Command G90 [ ],
 Command G17 [ ],
 Command G0 [ ],
 Command G0 [ X:132.14 Y:88.24 ],
 Command G1 [ X:132.14 Y:88.24 Z:0 ],
 Command G1 [ X:132.14 Y:-11.76 Z:0 ],
 Command G1 [ X:-67.863 Y:-11.76 Z:0 ],
 Command G1 [ X:-67.863 Y:88.24 Z:0 ],
 Command G1 [ X:132.14 Y:88.24 Z:0 ]]

In [12]:
import math
def pathcmdstosvg(cmds, bbox):
    dlist = [ ]
    prevX, prevY = None, None
    for cmd in cmds:
        X, Y = cmd.Parameters.get('X'), cmd.Parameters.get('Y')
        if X is None or Y is None:
            prevX, prevY = None, None
            continue
        elif cmd.Name == 'G1' and X == prevX and Y == prevY:
            continue
        
        if cmd.Name == 'G0':
            dlist.append("M%f %f" % (X, Y))
        elif cmd.Name == 'G1':
            dlist.append("L%f %f" % (X, Y))
        elif cmd.Name in ['G2', 'G3']:
            arcrad = math.hypot(cmd.Parameters.get('I'), cmd.Parameters.get('J'))
            largarcflag = 0
            sweepflag = 0 if cmd.Name == 'G2' else 1
            dlist.append("A%f %f 0 %d %d %f %f" % (arcrad, arcrad, largarcflag, sweepflag, X, Y))
        else:
            assert False, (cmd.Name, "not implemented")
        prevX, prevY = X, Y
        
        if "Xlo" not in bbox or X < bbox["Xlo"]:  bbox["Xlo"] = X
        if "Xhi" not in bbox or X > bbox["Xhi"]:  bbox["Xhi"] = X
        if "Ylo" not in bbox or Y < bbox["Ylo"]:  bbox["Ylo"] = Y
        if "Yhi" not in bbox or Y > bbox["Yhi"]:  bbox["Yhi"] = Y
            
    return "".join(dlist)



In [8]:
partfaces

In [13]:
partface = partfaces[0]
bbox = {}
pathcmdstosvg(Path.fromShapes(partface).Commands, bbox)

'M-23.112174 18.013507L-12.297852 12.027573L-34.866428 -11.760233L-57.980099 2.108736L-25.594664 9.179181L-63.651543 18.784637A10.433051 10.433051 0 0 0 -58.408873 37.550903L-23.112174 18.013507'

In [61]:
bbox = { }
svgstock = pathcmdstosvg(Path.fromShapes(stockface).Commands, bbox)
svgparts = [ pathcmdstosvg(Path.fromShapes(partface).Commands, bbox)  for partface in partfaces ]

fout = open("svgnestinput.svg", "w")
fout.write('<svg xmlns="http://www.w3.org/2000/svg" viewBox="%f %f %f %f">\n' % \
            (bbox["Xlo"], bbox["Ylo"], max(bbox["Xhi"]-bbox["Xlo"], 200), max(bbox["Yhi"]-bbox["Ylo"], 200)))
fout.write('<path id="stock" d="%s"/>\n' % svgstock)
for i, svgpart in enumerate(svgparts):
    fout.write('<path id="part%d" d="%s"/>\n' % (i, svgpart))
fout.write("</svg>\n")
fout.close()


In [15]:
# Then upload, run SVGnest and download to file
svgnestoutput = "/home/julian/Downloads/SVGnest-output(9).svg"

In [17]:
import re
svgtext = open(svgnestoutput).read()
svgtransforms = re.findall('<g transform="(.*?)"><path id="(.*?)"', svgtext)
assert svgtransforms[0] == ('translate(0 0)', 'stock'), svgtransforms[0]
#for (svgtransform, partid) in svgtransforms[1:]:
print(svgtransforms)


[('translate(0 0)', 'stock'), ('translate(11.760232899999993 -12.297851599999998) rotate(270)', 'part0'), ('translate(12.223722400000021 106.32086950000001) rotate(270)', 'part4'), ('translate(8.247282000000009 88.9164173) rotate(270)', 'part1'), ('translate(-5.295236600000003 82.8470767) rotate(270)', 'part2'), ('translate(118.3693485 93.90854999999999) rotate(180)', 'part5'), ('translate(9.1287737 111.82194589999999) rotate(0)', 'part3')]


In [21]:
m = re.search('class="bin" transform="translate\((.*?) (.*?)\)"', svgtext)
stocksx, stocksy = float(m.group(1)), float(m.group(2))
stocksx, stocksy

(67.8629379, 35.5110512)

In [18]:
import re
svgtext = open(svgnestoutput).read()
svgtransforms = re.findall('<g transform="(.*?)"><path id="(.*?)"', svgtext)
assert svgtransforms[0] == ('translate(0 0)', 'stock'), svgtransforms[0]

# actual structure of the stock is to stash a translate component in tail
m = re.search('class="bin" transform="translate\((.*?) (.*?)\)"', svgtext)
stocksx, stocksy = float(m.group(1)), float(m.group(2))

for svgtransform, partid in svgtransforms[1:]:
    m = re.match('translate\((.*?) (.*?)\) rotate\((.*?)\)', svgtransform)
    sx, sy, rot = float(m.group(1)), float(m.group(2)), float(m.group(3))
    i = int(partid[4:])
    print(i)
    job.Model.Group[i].Placement = Placement(Vector(sx,sy,0), Rotation(rot,0,0))


0
4
1
2
5
3


In [19]:
job.Model.Group

[<Part::PartFeature>, <Part::PartFeature>, <Part::PartFeature>]

In [88]:
doc.saveCopy("test2.fcstd")